# Controling Lab Equipment with Python Tutorial

We can use Python to control programmable lab equipment instead of using proprietary software such as NI-MAX or LABVIEW.

## Libraries

* PyVISA
* nidaqmx
* threading

### PyVISA (https://pyvisa.readthedocs.io/en/latest/)

The PyVISA library is a Python wrapper for the VISA library which already exists for C, Visual Basic, and G (LABVIEW). It allows you to communicate with instrumentation systems which use USB, ethernet, GPIB, VXI, PXI, or serial interfaces [1]. While the specific programing commands will be dependent on the device, PyVISA provides a library to communicate with any devices which can be connected to the computer. For example, we can control a Tektronix Oscilloscope to read the screen.


This particular oscilloscope is the Tektronix MDO34, for any given programmable device you will need to know its VISA communication ID and its programming commands. IEEE 488.2 mandated specific protocol and data formats but not command interfaces. To fix this problem the Standard Commands for Programmable Instruments (SCPI) was developed. All programmable devices will use at a minimum, the common commands from this set, in addition to following the required command format. I will breifly go over the command format, though all programming manuals will also do that.

A command can either perform an operation (set) or request information (query), in either case it will take the following form:
```
[:]Header:[Subfunction] Argument[;]
```
The semicolon at the end is optional but allows you to chain commands in a single line. Normally the colon at the beginning is optional, but when chaining commands, it is required:
```
:Header:[Subfunction] Argument; :Header;[Subfunction] Argument
```
The argument is only for set operations which require it. A query does not have an argument and takes the form:
```
[:]Header:[Subfunction]?
```
and will get a return in the form
```
:Header:[Subfunction] value[;]
```
If multiple queries are chained with semicolons, so will the output.

Some functions have subfunctions within them, if they do, that is when the optional subfunction section is used.

The final command syntax rule to know is that commands can be shortened based on their capitalized letters. For example, Aquisition commands for an oscilloscope are aquired and processed into waveforms. The aquire header is found in the programming manual to be ```ACQuire``` and the acquisition mode is ```MODe``` with possible arguments of: ```SAMple```, ```PEAKdetect```, ```HIRes```, ```AVErage```, ```ENVelope```. While I wont go into detail on the meaning of each of those, if I wanted to acquire data from an oscilloscope input in Hi Res mode I would write:

```
ACQuire:MODe HIRes
```

Or to query the mode I can write:

```
ACQuire:MODe?
```
These can both be shortened by only using the capitalized words:
```
ACQ:MOD HIR
ACQ:MOD?
```

For specific device commands I recommend to find a "programming manual" for the device, which usually can be found at the manufacturer's website. In this case we have this one: https://download.tek.com/manual/3-MDO-Oscilloscope-Programmer-Manual-077149800.pdf.

### Example

To start, we can install the needed libraries, your computer will also need USB Test & Measurement Class specification (USBTMC) to be able to communicate over USB. This is not an issue for Windows machines, but Linux, and especially MACOS will struggle.

In [8]:
pip install pyvisa

Note: you may need to restart the kernel to use updated packages.


You will also likely need the backend libraries: pyvisa-py will provide the background for the visa communication and zeroconf will help with more advanced resource discovery.

In [14]:
pip install pyvisa-py

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install zeroconf

Note: you may need to restart the kernel to use updated packages.


Next we can import the library

In [8]:
import pyvisa

And now we can start to communicate with devices. To see VISA compatible devices connected to your computer you will need to first create a resource manager. The resource manager allows you to see, open, and communicate with VISA devices.

In [11]:
rm = pyvisa.ResourceManager()

Once you have done that, you can find the current connected devices with the ```list_resources``` method

In [12]:
rm.list_resources()


()

At this point we can open the device we want. When we open the device we will want to assign it to an object to send it commands later. We can open a device with the resrouce manager method ```open_resource('resource name')```, where the resource name is the string listed in the above list which we want to use.

In [20]:
oscilloscope = rm.open_resource(rl[0])

IndexError: tuple index out of range

Once we have a device we can query to get its name and make sure that we are working on the right device. The ```*IDN``` command is a common command which all VISA devices will understand. There a number of other common commands denoted by an *, they can be found in any device programming manual. When sending these common commands, regardless of if they are in series with a semi-colon, they do not have a colon to start.

In [15]:
oscilloscope.query('*IDN?')

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [4]:
oscilloscope.close()
rm.close()

NameError: name 'oscilloscope' is not defined

## NI-DAQmx (https://nidaqmx-python.readthedocs.io/en/stable/#installation)


In the case of National Instruments (NI) Data Aquisition Devices (DAQs), we cannot use VISA as the devices use a different comunnication API.The nidaqmx python package allows you to communicate with NI DAQs through Python on Windows or Linux machines, (MAC devices may work but will require a lot more effort with installing correct drivers and may not be fully compatable). It requires the computer to have the NI-DAQmx driver installed in order to identify and communicate with the device. If you have the correct drivers and a physical or virtual NI-DAQ is connected to the computer, then you are able to program it using Python.

To do this we will first install the library

In [1]:
pip install nidaqmx

Note: you may need to restart the kernel to use updated packages.


Then import

In [4]:
import nidaqmx

At this point we can connect and control an NI-DAQ. Using this library, all commands you send to a DAQ are done using a ```Task()``` object. This object is a collection of channels which represent the measurements you want to take. For most devices, you will only have one task, thus it is reasonable to use:

In [6]:
task = nidaqmx.Task()
task.close()

DaqNotSupportedError: NI-DAQmx Python is not supported on this platform: darwin. Please direct any questions or feedback to National Instruments.

This task will then be open forever until you close it, not closing the task can cause issues; thus, it can be easier to do this:

In [7]:
with nidaqmx.Task() as task:
    pass

DaqNotSupportedError: NI-DAQmx Python is not supported on this platform: darwin. Please direct any questions or feedback to National Instruments.

which automatically closes the task when the internal code is completed.

Great, now we need to control the DAQ. To do this, we must open a channel. A virtual channel in Python here corresponds to a physical port on your DAQ. There are four types of channels in the nidaqmx ```Task()```: analog in, analog out, digital in, and digital out. We can create these channels if we know the name of the DAQ and what physical channel we want to use. We can also designate acceptable ranges, terminal configuration, or scaling. 

For example, here we create two analog input channels, one using differential mode and the other using referenced single ended (ie, reference to ground).

In [ ]:
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("DAQ/ai0", min_val=0, max_val=10, terminal_config=nidaqmx.TerminalConfiguration.DIFF)
    task.ai_channels.add_ai_voltage_chan("DAQ/ai1", min_val=0, max_val=10, terminal_config=nidaqmx.TerminalConfiguration.RSE)

## Threading

References

[1] https://pyvisa.readthedocs.io/en/latest/

[2] https://nidaqmx-python.readthedocs.io/en/stable/#installation
